In [ ]:
import sys
import requests
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QLineEdit, QPushButton, QComboBox
from PyQt6.QtGui import QFont, QIcon
from PyQt6.QtCore import Qt

# API Constants
API_KEY = "0fe457368e8f6620e2157c473dceedcf"  # Replace with your actual API key
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"

LANGUAGES = {"English": "en", "French": "fr", "Spanish": "es", "Kinyarwanda": "rw"}

def convert_temperature(value, from_unit, to_unit):
    conversions = {
        ('C', 'F'): lambda x: (x * 9/5) + 32,
        ('F', 'C'): lambda x: (x - 32) * 5/9,
        ('C', 'K'): lambda x: x + 273.15,
        ('K', 'C'): lambda x: x - 273.15,
        ('F', 'K'): lambda x: (x - 32) * 5/9 + 273.15,
        ('K', 'F'): lambda x: (x - 273.15) * 9/5 + 32,
    }
    return conversions.get((from_unit, to_unit), lambda x: x)(value)

def get_weather(city, temp_unit="C", lang="en"):
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": lang}
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        temperature = data["main"]["temp"]
        if temp_unit != "C":
            temperature = convert_temperature(temperature, "C", temp_unit)
        unit = "°C" if temp_unit == "C" else "°F" if temp_unit == "F" else "K"
        
        return (
            f"City: {data['name']}\n"
            f"Temperature: {temperature:.2f}{unit}\n"
            f"Humidity: {data['main']['humidity']}%\n"
            f"Pressure: {data['main']['pressure']} hPa\n"
            f"Wind Speed: {data['wind']['speed']} m/s\n"
            f"Visibility: {data.get('visibility', 'N/A')} meters\n"
            f"Cloud Cover: {data['clouds']['all']}%\n"
            f"Sunrise: {data['sys']['sunrise']}\n"
            f"Sunset: {data['sys']['sunset']}\n"
            f"Description: {data['weather'][0]['description'].capitalize()}"
        )
    else:
        return "Error: City not found or API issue."

def get_forecast(city, temp_unit="C", lang="en"):
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": lang}
    response = requests.get(FORECAST_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        forecast_text = "5-Day Forecast:\n"
        
        for i in range(0, len(data['list']), 8):  # Every 8th entry corresponds to a new day
            day_data = data['list'][i]
            date = day_data['dt_txt'].split(" ")[0]
            temp = day_data['main']['temp']
            if temp_unit != "C":
                temp = convert_temperature(temp, "C", temp_unit)
            desc = day_data['weather'][0]['description'].capitalize()
            forecast_text += f"{date}: {temp:.2f}°{temp_unit}, {desc}\n"
        
        return forecast_text
    else:
        return "Error: Forecast data not available."

class WeatherApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Weather App")
        self.setGeometry(100, 100, 400, 450)
        
        # Set the window icon
        self.setWindowIcon(QIcon("weather_icon.png"))  # Ensure the icon file is in the same directory
        
        layout = QVBoxLayout()

        self.city_input = QLineEdit(self)
        self.city_input.setPlaceholderText("Enter city name")
        layout.addWidget(self.city_input)

        self.unit_selector = QComboBox(self)
        self.unit_selector.addItems(["Celsius (°C)", "Fahrenheit (°F)", "Kelvin (K)"])
        layout.addWidget(self.unit_selector)

        self.language_selector = QComboBox(self)
        self.language_selector.addItems(LANGUAGES.keys())
        layout.addWidget(self.language_selector)

        self.get_weather_btn = QPushButton("Get Weather", self)
        self.get_weather_btn.clicked.connect(self.fetch_weather)
        layout.addWidget(self.get_weather_btn)

        self.result_label = QLabel("Weather info will appear here", self)
        self.result_label.setFont(QFont("Arial", 12))
        self.result_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.result_label)
        
        self.forecast_label = QLabel("5-Day Forecast will appear here", self)
        self.forecast_label.setFont(QFont("Arial", 12))
        self.forecast_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.forecast_label)

        self.setLayout(layout)

    def fetch_weather(self):
        city = self.city_input.text().strip()
        unit = self.unit_selector.currentText()[0]  # Get first letter (C, F, or K)
        lang = LANGUAGES[self.language_selector.currentText()]
        if city:
            weather_info = get_weather(city, unit, lang)
            forecast_info = get_forecast(city, unit, lang)
            self.result_label.setText(weather_info)
            self.forecast_label.setText(forecast_info)
        else:
            self.result_label.setText("Please enter a valid city name.")
            self.forecast_label.setText("")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    
    # Set global app icon (optional)
    app.setWindowIcon(QIcon("weather_icon.png"))
    
    window = WeatherApp()
    window.show()
    sys.exit(app.exec())
